In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold, train_test_split

from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
from sklearn.neural_network import MLPClassifier



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')

In [ ]:
print('No Frauds', round(df['Class'].value_counts()[0]/len(df) * 100,2), '% of the dataset')
print('Frauds', round(df['Class'].value_counts()[1]/len(df) * 100,2), '% of the dataset')

In [ ]:
rob_scaler = RobustScaler()

df['scaled_amount'] = rob_scaler.fit_transform(df['Amount'].values.reshape(-1,1))
df['scaled_time'] = rob_scaler.fit_transform(df['Time'].values.reshape(-1,1))

df.drop(['Time','Amount'], axis=1, inplace=True)

In [ ]:
scaled_amount = df['scaled_amount']
scaled_time = df['scaled_time']

df.drop(['scaled_amount', 'scaled_time'], axis=1, inplace=True)
df.insert(0, 'scaled_amount', scaled_amount)
df.insert(1, 'scaled_time', scaled_time)

df.head()

In [ ]:
sm = SMOTE(random_state=42)
X_hat, y_hat = sm.fit_resample(df.drop('Class', axis=1), df['Class'])

total_df = pd.concat([X_hat, y_hat], axis=1)
total_df = total_df.sample(frac=1)

total_df.head()

In [ ]:
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(total_df.drop('Class', axis=1))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, total_df['Class'], test_size=0.2, random_state=42)

In [ ]:
clf = MLPClassifier(random_state=1, max_iter=200, verbose=True, batch_size=100).fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)